### Libraries Allowed
* pandas, scipy, numpy
* sklearn
* datetime
* dateutil
* prophet
* patsy (for model matrix), statsmodels
* xgboost
* warnings

In [1]:
import sys
import time
import pandas as pd
import numpy as np
import scipy
from scipy.stats.mstats import winsorize
import datetime
import dateutil
from tqdm import tqdm
import warnings

import statsmodels
import xgboost
import prophet
import patsy

from datetime import datetime, timedelta
import statsmodels.api as sm
import time

Importing plotly failed. Interactive plots will not work.


In [2]:
warnings.filterwarnings('ignore')

In [8]:
def myeval(num_folds=10):
    file_path = 'Proj2_Data/test_with_label.csv'
    test_with_label = pd.read_csv(file_path)
    #num_folds = 10
    wae = []

    for i in range(num_folds):
        file_path = f'Proj2_Data/fold_{i+1}/test.csv'
        test = pd.read_csv(file_path)
        test = test.drop(columns=['IsHoliday']).merge(test_with_label, on=['Date', 'Store', 'Dept'])

        file_path = f'Proj2_Data/fold_{i+1}/mypred.csv'
        test_pred = pd.read_csv(file_path)

        # Left join with the test data
        new_test = test_pred.merge(test, on=['Date', 'Store', 'Dept'], how='left')

        # Compute the Weighted Absolute Error
        actuals = new_test['Weekly_Sales']
        preds = new_test['Weekly_Pred']
        weights = new_test['IsHoliday'].apply(lambda x: 10 if x else 1)
        wae.append(sum(weights * abs(actuals - preds)) / sum(weights))

    for value in wae:
        print(f"wae_by_fold={value:.3f}")
    print(f"overall wae={sum(wae) / len(wae):.3f}")
    
    # print(wae)
    # print(np.mean(wae))    
        
    return wae

In [9]:
def preprocess(data):
    tmp = pd.to_datetime(data['Date'])
    data['Wk'] = tmp.dt.isocalendar().week
    data['Yr'] = tmp.dt.year
    data['Wk'] = pd.Categorical(data['Wk'], categories=[i for i in range(1, 53)])  # 52 weeks 
#    data['IsHoliday'] = data['IsHoliday'].apply(int)
    return data


In [5]:
def pca_smooth_train(train):
    smooth_dept_trains = []
    departments = train['Dept'].unique()

    for department in departments:
        # Filter rows where Dept is equal to 1
        filtered_train = train[train['Dept'] == department]
        # Select only the columns 'Store', 'Date', and 'Weekly_Sales'
        selected_columns = filtered_train[['Store', 'Date', 'Weekly_Sales']]
        # Pivot table to spread 'Store' values into columns, with 'Weekly_Sales' as values
        train_dept_ts = selected_columns.pivot(index='Date', columns='Store', values='Weekly_Sales').reset_index()

        X_train = train_dept_ts.iloc[:, 1:]

        # Smooth department data
        X_train = X_train.to_numpy()
        X_train = np.nan_to_num(X_train)
        store_means = np.mean(X_train, axis=0)
        X_train = X_train - store_means
        X_train = np.transpose(X_train)

        U, D, V_t = np.linalg.svd(X_train, full_matrices=False)
        D[8:] = 0
        F_train = U @ np.diag(D) @ V_t

        stores_list = train_dept_ts.columns[1:]
        F_train = pd.DataFrame(np.transpose(F_train), columns=stores_list)
        F_train = F_train.add(store_means, axis=1)
        F_train["Date"] = train_dept_ts["Date"]

        smooth_dept_train = pd.melt(F_train, id_vars=['Date'], value_vars = stores_list, \
                                    var_name='Store', value_name='Weekly_Sales')
        smooth_dept_train["Dept"] = department
        smooth_dept_train["Store"] = smooth_dept_train["Store"].astype(np.int64)
        smooth_dept_trains.append(smooth_dept_train)

    smooth_train = pd.concat(smooth_dept_trains, ignore_index=True)
    return smooth_train


In [6]:
%%time

num_folds = 10

for j in tqdm(range(1, num_folds + 1)):
    start_time = time.time()

    # Reading train data
    file_path = f"Proj2_Data/fold_{j}/train.csv"
    train = pd.read_csv(file_path)

    # Smooth train data
    smoothed = pca_smooth_train(train)

    train_dupe = train[["Date", "IsHoliday"]].drop_duplicates()
    train = smoothed.merge(train_dupe, on=["Date"], how="left")

    # Reading test data
    file_path = f"Proj2_Data/fold_{j}/test.csv"
    test = pd.read_csv(file_path)

    # pre-allocate a pd to store the predictions
    test_pred = pd.DataFrame()

    train_pairs = train[["Store", "Dept"]].drop_duplicates(ignore_index=True)
    test_pairs = test[["Store", "Dept"]].drop_duplicates(ignore_index=True)
    unique_pairs = pd.merge(
        train_pairs, test_pairs, how="inner", on=["Store", "Dept"]
    )

    train_split = unique_pairs.merge(train, on=["Store", "Dept"], how="left")
    train_split = preprocess(train_split)
    y, X = patsy.dmatrices(
        "Weekly_Sales ~ Weekly_Sales + Store + Dept + Yr + np.power(Yr, 2)  + Wk",
        data=train_split,
        return_type="dataframe",
    )
    train_split = dict(tuple(X.groupby(["Store", "Dept"])))

    test_split = unique_pairs.merge(test, on=["Store", "Dept"], how="left")
    test_split = preprocess(test_split)
    y, X = patsy.dmatrices(
        "Yr ~ Store + Dept + Yr + np.power(Yr, 2) + Wk", data=test_split, return_type="dataframe"
    )
    X["Date"] = test_split["Date"]
    test_split = dict(tuple(X.groupby(["Store", "Dept"])))

    keys = list(train_split)

    for key in keys:
        X_train = train_split[key]
        X_test = test_split[key]

        Y = X_train["Weekly_Sales"]
        X_train = X_train.drop(["Weekly_Sales", "Store", "Dept"], axis=1)

        model = sm.OLS(Y, X_train).fit()
        mycoef = model.params.fillna(0)

        tmp_pred = X_test[["Store", "Dept", "Date"]]
        X_test = X_test.drop(["Store", "Dept", "Date"], axis=1)

        tmp_pred["Weekly_Pred"] = np.dot(X_test, mycoef)
        test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)

    test_pred["Weekly_Pred"].fillna(0, inplace=True)

    # Post-prediction adjustment for fold 5
    if j == 5:
        dates = pd.to_datetime(test_pred["Date"])
        test_pred["Wk"] = dates.dt.isocalendar().week

        test_pred_51 = test_pred[test_pred["Wk"] == 51]
        test_pred_51["Shift"] = test_pred_51["Weekly_Pred"] / 9
        test_pred_52 = test_pred[test_pred["Wk"] == 52]

        test_pred_52 = test_pred_52.merge(
            test_pred_51[["Store", "Dept", "Shift"]],
            on=["Store", "Dept"], how="left"
        )

        test_pred_51 = test_pred_51.merge(
            test_pred_52[["Store", "Dept"]],
            on=["Store", "Dept"], how="left", indicator=True
        )
        test_pred_51[test_pred_51["_merge"] == "left_only"]["Shift"] = 0

        test_pred_52["Date"] = "2011-12-30"
        test_pred_52["Shift"].fillna(0, inplace=True)
        test_pred_52["Weekly_Pred"].fillna(0, inplace=True)

        # test_pred_51["Weekly_Pred"] = test_pred_51["Weekly_Pred"] - test_pred_51["Shift"]
        test_pred_52["Weekly_Pred"] = test_pred_52["Weekly_Pred"] + test_pred_52["Shift"]

        test_pred_51.drop("Shift", inplace=True, axis=1)
        test_pred_52.drop("Shift", inplace=True, axis=1)

        test_pred = test_pred[(test_pred["Wk"] != 51) & (test_pred["Wk"] != 52)]
        test_pred = pd.concat([test_pred, test_pred_51, test_pred_52], ignore_index=True)
        test_pred.drop(columns=["Wk"], inplace=True)

    # Save the output to CSV
    file_path = f"Proj2_Data/fold_{j}/mypred.csv"
    test_pred.to_csv(file_path, index=False)
    print(file_path)

#     print(
#         "--- fold {fold}, {wae_fold}, {elapsed} seconds ---".format(
#             fold=j,
#             wae_fold=f"{myeval_fold(j, test_with_label):.3f}",
#             elapsed=time.time() - start_time
#         )
#     )


 10%|███████████▊                                                                                                          | 1/10 [00:07<01:08,  7.58s/it]

Proj2_Data/fold_1/mypred.csv


 20%|███████████████████████▌                                                                                              | 2/10 [00:17<01:09,  8.73s/it]

Proj2_Data/fold_2/mypred.csv


 30%|███████████████████████████████████▍                                                                                  | 3/10 [00:25<01:01,  8.79s/it]

...

Proj2_Data/fold_8/mypred.csv


 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▏           | 9/10 [01:49<00:14, 14.15s/it]

Proj2_Data/fold_9/mypred.csv


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:03<00:00, 12.30s/it]

Proj2_Data/fold_10/mypred.csv
CPU times: user 11min 5s, sys: 5min 5s, total: 16min 11s
Wall time: 2min 3s


In [7]:
%%time

wae = myeval()


wae_by_fold=1945.895
wae_by_fold=1364.090
wae_by_fold=1384.009
wae_by_fold=1528.647
wae_by_fold=2220.198
wae_by_fold=1636.200
wae_by_fold=1614.822
wae_by_fold=1355.266
wae_by_fold=1337.461
wae_by_fold=1334.495
overall wae=1572.108
CPU times: user 803 ms, sys: 502 ms, total: 1.31 s
Wall time: 525 ms
